In [2]:
import os
from utils.auxiliary_func import *
from utils.plots_func import *
import config.const as const
import config.params as params
import tifffile 

### Define Parameters

In [13]:
leap_num = '043'

In [3]:
figure_1_dir = os.path.join(const.BASE_DIR, 'figure_results', 'Figure 1')
os.makedirs(figure_1_dir, exist_ok=True)

DATA_PREPARATION_DIR = const.DATA_PREPARATION_DIR

## Create FLIM Image processing

### Create segmentation map - with black background and gray for the segmentation labels

In [15]:
df_seg_file_path = os.path.join(const.SEG_DIR, f"LEAP{leap_num}_segmentation_labels.tif")
img_seg = Image.open(df_seg_file_path)
img_seg = np.array(img_seg)

binary_original = (img_seg > 0).astype(np.uint8)
H, W = binary_original.shape
rgb = np.zeros((H, W, 3), dtype=np.uint8)      # black background
rgb[binary_original == 1] = [80, 80, 80]    # gray foreground

# Define the output path
output_seg_path  = os.path.join(figure_1_dir , f"LEAP{leap_num}_seg_binary_values.tif") 

# Save the array as a TIFF file
tifffile.imwrite(output_seg_path, rgb)


### Create quality control segmentation map - with remove and remain regions

In [16]:
df_seg_qc_file_path = os.path.join(const.SEG_AFTER_QC_DIR, f"LEAP{leap_num}_segmentation_labels_qc.tif") 
img_seg_qc = Image.open(df_seg_qc_file_path)
img_seg_qc = np.array(img_seg_qc)

df_seg_file_path = os.path.join(const.SEG_DIR, f"LEAP{leap_num}_segmentation_labels.tif")
img_seg = Image.open(df_seg_file_path)
img_seg = np.array(img_seg)

binary_original = (img_seg > 0).astype(np.uint8)
binary_qc = (img_seg_qc > 0).astype(np.uint8)

# Multiply to find overlapping areas (retained)
mask_retained = binary_original * binary_qc  # 1 where label retained
# Subtract to find removed labels (present in original but not in QC)
mask_removed = binary_original * (1 - binary_qc)  # 1 where label removed

# Create RGB image for visualization
highlight_binary = np.zeros((*img_seg.shape, 3), dtype=np.uint8)
highlight_binary[mask_removed == 1] = [255, 165, 0]  # Orange for removed
highlight_binary[mask_retained == 1] = [0, 0, 255]   # Blue for retained


output_qc_path = os.path.join(figure_1_dir , f"LEAP{leap_num}quality_control.tif") 
tifffile.imwrite(output_qc_path, highlight_binary)


### Create nucleus single lifetime value image

#### Note on Data Preparation

The sub_features_df.csv used in this code was generated in the following notebook:

**`TNBC_FLIM/notebooks/create_sub_features_df.ipynb`**

In [22]:
sub_features_path = os.path.join(DATA_PREPARATION_DIR, "sub_features_df.csv")
full_tissue_features_df = pd.read_csv(sub_features_path, dtype={'leap_ID':str})

#### Create the image

In [17]:
labels  = tifffile.imread(
    os.path.join(const.SEG_AFTER_QC_DIR, f"LEAP{leap_num}_segmentation_labels_qc.tif")).astype(np.int32)

# Filter probabilities
leap_probs = full_tissue_features_df[full_tissue_features_df["leap_ID"] == leap_num].copy()

max_label = labels.max()
lookup = np.full(max_label + 1, np.nan, dtype=np.float32)

label_ids = leap_probs["nucleus_label"].astype(np.int32).values
lifetime_vals = leap_probs["lifetime_mean"].astype(np.float32).values
lookup[label_ids] = lifetime_vals
lookup = np.nan_to_num(lookup, nan=0.0)

lifetime_map = lookup[labels] 

single_nuclei_lifetime_path = os.path.join(figure_1_dir, f'Leap{leap_num}_single_lifetime.tif')

tifffile.imwrite(single_nuclei_lifetime_path, lifetime_map.astype(np.float32))

## Lifetime median & distribution

### Use median data frame - core samples

In [4]:
df_median_file_path = os.path.join(const.FULL_TISSUE_DIR, 'core', "features_median_data.csv")
median_df = pd.read_csv(df_median_file_path, dtype={'leap_ID': str})
median_df.head(2)

,leap_ID,lifetime_mean,area,extent,solidity,perimeter,diameter_area,convex_hull_area,minor_axis_length,perimeter_crofton,major_axis_length,orientation,diameter_max,eccentricity,density_radius_20,density_radius_40,density_radius_60,density_radius_80,category
0,015,3.915147,16.0,0.75,0.954545,12.242641,4.513517,17.0,3.669612,14.518306,5.449354,0.000000,5.385165,0.713180,0.007958,0.005769,0.005040,0.004526,non responder
1,017,4.177300,18.0,0.75,0.944444,13.656854,4.787307,20.0,3.991464,16.184387,6.065440,0.084918,6.324555,0.722862,0.019099,0.018104,0.016977,0.016015,non responder


In [5]:
median_core_path = os.path.join(DATA_PREPARATION_DIR, "features_median_data_core.csv")
median_df.to_csv(median_core_path, index=False)

## Create results data frame of lifetime distribution model - tissue wise

### Define Parameters

In [5]:
fold_method = 'loocv'
model_name = 'xgboost'
tissue_resulotion = 'tissue_wise'
feature_type = 'lifetime'
bin_amount = 18

In [8]:
print(f"Create results data frame of bin nums : {bin_amount}")
distribution_dir = os.path.join(const.DISTRIBUTION_RESULTS_FULL_TISSUE_DIR, feature_type, model_name, fold_method, f"{bin_amount}_bins")
seeds, auc_list, fpr_list, tpr_list = extract_distribution_seed_results(distribution_dir)

data = {
    'seed_val': seeds,
    'auc': auc_list,
    'fpr': [f.tolist() for f in fpr_list],
    'tpr': [t.tolist() for t in tpr_list],
}

results_df = pd.DataFrame(data)
results_df['seed_val'] = results_df['seed_val'].str.replace('seed_', '').astype(int)

print(results_df.head(2))

model_results_name = f'bins_{bin_amount}_lifetime_distribution_model_results_tissue_wise.csv'
save_path = os.path.join(DATA_PREPARATION_DIR, model_results_name)

results_df.to_csv(save_path, index=False)
print(f"CSV file of lifetime distribution with {bin_amount} bins saved\n")

Create results data frame of bin nums : 18
   seed_val       auc                                                fpr  \
0      1056  0.714493  [0.0, 0.043478260869565216, 0.0434782608695652...   
1       107  0.720290  [0.0, 0.043478260869565216, 0.0434782608695652...   

                                                 tpr  
0  [0.0, 0.0, 0.3333333333333333, 0.3333333333333...  
1  [0.0, 0.0, 0.1, 0.1, 0.13333333333333333, 0.13...  
CSV file of lifetime distribution with 18 bins saved

